In [ ]:
"""
NOTE: Before running this, you'll want to do the following setup:

ipython

%run preprocess.py
make_images("png", 640)
preprocess_csv()
make_id_orig_size_csv()

%run make_data_class.py
create_and_validate_data_folds(src="png640", dst="png640_3fold", num_folds=3, extn="png")

!pip install albumentations fastbook timm==0.4.12

"""

In [1]:
from fastai.vision.all import *
import pickle
from importlib import reload

import const
const = reload(const)
import train_class
train_class = reload(train_class)

"""
vastai prep:
- pip install fastai[all]
"""

'\nvastai prep:\n- pip install fastai[all]\n'

In [ ]:
train_class.make_base_models(
    [
        "tf_efficientnet_b3", 
        "tf_efficientnet_b3_ap",
        "tf_efficientnet_b3_ns",
    ]
)

In [3]:
from typing import List
IMAGE_PATH = "jpg640_3fold"

def run_test(model_name, image_size, num_epochs, lr):
    print(f"\nModel: {model_name}, Img size: {image_size}, epochs: {num_epochs}, lr: {lr}")
    
    dls = train_class.get_dls(
        image_path=IMAGE_PATH, 
        img_size=image_size, 
        is_neg=False, 
        fold_valid=0,
    )
    
    learn = train_class.get_learn(dls=dls, model_name=model_name, is_neg=False)
    
    res = learn.lr_find()
    plt.pause(0.001)
    
    learn.fit_one_cycle(num_epochs, lr)
    plt.pause(0.001)
    
    preds, targs = train_class.predict_and_save_folds(
        learn_folds=[learn], 
        model_name=model_name, 
        n_tta=6,
        is_neg=False,
    )
    print("6 TTA:", train_class.preds_map(preds, targs))

def run_tests(tests, lrs):
    for test, lrs_i in zip(tests, lrs):
        if not isinstance(lrs_i, List):
            lrs_i = [lrs_i]
        for lr in lrs_i:
            run_test(*test, lr=lr)

In [7]:
tests = list(itertools.product(
    [
        "tf_efficientnet_b3", 
        "tf_efficientnet_b3_ap",
        "tf_efficientnet_b3_ns",
    ],
    [368, 512, 640],
    [5, 10],
))
tests

[('tf_efficientnet_b3', 368, 5),
 ('tf_efficientnet_b3', 368, 10),
 ('tf_efficientnet_b3', 512, 5),
 ('tf_efficientnet_b3', 512, 10),
 ('tf_efficientnet_b3', 640, 5),
 ('tf_efficientnet_b3', 640, 10),
 ('tf_efficientnet_b3_ap', 368, 5),
 ('tf_efficientnet_b3_ap', 368, 10),
 ('tf_efficientnet_b3_ap', 512, 5),
 ('tf_efficientnet_b3_ap', 512, 10),
 ('tf_efficientnet_b3_ap', 640, 5),
 ('tf_efficientnet_b3_ap', 640, 10),
 ('tf_efficientnet_b3_ns', 368, 5),
 ('tf_efficientnet_b3_ns', 368, 10),
 ('tf_efficientnet_b3_ns', 512, 5),
 ('tf_efficientnet_b3_ns', 512, 10),
 ('tf_efficientnet_b3_ns', 640, 5),
 ('tf_efficientnet_b3_ns', 640, 10)]

In [ ]:
run_tests(tests, lrs=[[7e-4, 1e-3, 2e-3]]*len(tests))